#Tarea 2/ parte 2 Métodos numéricos
##Leslie Janeth Quincosa Ramírez

#Ejercicio 4
 El sistema lineal dado por

$$
\begin{aligned}
x_{1}+2 x_{2} & =3 \\
1.0001 x_{1}+2 x_{2} & =3.0001
\end{aligned}
$$

tiene como solución el vector $(1,1)^{T}$. La matriz $A$ es obtenida mediante datos experimentales y la nueva medición cambia $A$ ligeramente, siendo el nuevo sistema

$$
\begin{aligned}
x_{1}+2 x_{2} & =3 \\
0.9999 x_{1}+2 x_{2} & =3.0001
\end{aligned}
$$

(a) Calcular la nueva solución usando aritmética de 5 dígitos y comparar el error obtenido con el estimado por:

$$
\frac{\|\hat{x}-x\|}{\|x\|} \leq \frac{\kappa(A)}{1-\kappa(A)(\|\delta A\| /\|A\|)}\left(\frac{\|\delta A\|}{\|A\|}+\frac{\|\delta b\|}{\|b\|}\right)
$$

donde $A$ y $b$ son tales que el sistema se escribe de la forma $A x=b$; $\delta A, \delta b$ son las perturbaciones asociadas a $A$ y $b ; \kappa(A)=\|A\|\left\|A^{-1}\right\|$. ¿Es la matriz A mal condicionada? Si es así realiza un tratamiento para obtener un sistema equivalente bien condicionado.

In [140]:
import numpy as np

In [141]:
A_original = np.array([[1,2],[1.0001, 2]], dtype = 'float64')
b_original = np.array([3, 3.0001], dtype = 'float64')

In [142]:
#Solución del sistema original
x= np.linalg.solve(A_original, b_original)
print(x)

[1. 1.]


In [143]:
#Inversa de la matriz A
Ainv = np.linalg.inv(A_original)
print(Ainv)

[[-10000.   10000. ]
 [  5000.5  -5000. ]]


In [144]:
#Número de la condición de la matriz A original
Ainv_norm = np.linalg.norm(Ainv, ord=np.inf)
A_norm = np.linalg.norm(A_original, ord=np.inf)

kA = A_norm*Ainv_norm

KA = np.linalg.cond(A_original, p=np.inf)
print(KA)

60001.99999999003


Notemos que el número de condición de $A$ es muy grande. Vamos a aplicar algún método que mejore esa condición

In [145]:
#Alteración a la matriz original, esta alteración es la que nos indica el problema

A_delta = np.array([[0, 0],[-0.0002, 0]], dtype = 'float64')
b_delta = np.array([0, 0], dtype = 'float64')

In [146]:
A_new = A_original+A_delta
b_new = b_original+b_delta

In [147]:
#Matriz alterada del ejercicio

A_new

array([[1.    , 2.    ],
       [0.9999, 2.    ]])

In [148]:
KA_new = np.linalg.cond(A_original+A_delta, p=np.inf)

In [149]:
#Solución del nuevo sistema
x_new = np.linalg.solve(A_new, b_new)
print(x_new)

[-1.  2.]


In [150]:
#Error de las normas del vector solución x
Error_1 = np.linalg.norm(x_new- x, ord=np.inf)/np.linalg.norm(x, ord=np.inf)
print("||x_hat - x||/||x||: ", Error_1)

#Error de las matriz de A y su alteración
Error_A = np.linalg.norm(A_delta, ord=np.inf)/np.linalg.norm(A_original, ord=np.inf)
print("||A_delta||/||A||: ",Error_A)

#Error de la norma de b
Error_b = np.linalg.norm(b_delta, ord=np.inf)/np.linalg.norm(b_original, ord=np.inf)
print("||b_delta||/||b||: ",Error_A)

||x_hat - x||/||x||:  2.0000000000044404
||A_delta||/||A||:  6.666444451851606e-05
||b_delta||/||b||:  6.666444451851606e-05


In [151]:
Error_2 = (KA/(1-KA*Error_A))*(Error_A + Error_b)
print("El segundo termino de la desigualdad: ", Error_2)

El segundo termino de la desigualdad:  -1.3333333333334072


Vimos que la matriz alterada del problema está mal condicionada. Por lo que mediante la factorización $QR$ encontramos que podemos llevar al sistema $Ax = b$ al sistema $MAx=Mb$ donde $M=QA^{-1}$, es decir, $Qx =QA^{-1}b$ de modo que ahora analicemos el sistema con está equivalencia.

In [152]:
#Descomposicion Grand-Schmitd QR

Q, R = np.linalg.qr(A_original)
#Ahora el sistema es Q = Q Ainv b x

x_GS= np.linalg.solve(Q, Q@Ainv@b_original)
print("Solución del sismtema Qx=b: ",x_GS)

#Número de condición de Q

KQ = np.linalg.cond(Q, p=np.inf)
print("El número de condición de Q : ", KQ)

print("||x_hat - x||/||x||: ", np.linalg.norm(x - x_GS, ord=np.inf))



Solución del sismtema Qx=b:  [1. 1.]
El número de condición de Q :  1.9999999950004999
||x_hat - x||/||x||:  1.1035616864774056e-12


#Ejercicio 5.

  Considere el sistema de ecuaciones

$$
\begin{array}{r}
3 x_{1}-x_{2}+x_{3}=1 \\
3 x_{1}+6 x_{2}+2 x_{3}=0 \\
3 x_{1}+3 x_{2}+7 x_{3}=4
\end{array}
$$

(a) Implementar el método SOR a partir de la expresión del método de Gauss-Seidel o Gauss-Jacobi, considera $\omega=\frac{2}{1+\sqrt{1-\rho(T)^{2}}}$ donde $T=D^{-1}(L+U)$ y da valores para $\omega$ tales que se tenga subrelajación o sobrerelajación. ¿Qué parámetro de relajación escogerías para una convergencia más rápida? Explica por qué.

(b) Tomando como valores iniciales $x^{0}=(0,0,0)^{T}$, comparar con las tres primeras iteraciones del método de sobrerelajación y el que has escogido como base (Gauss-Jacobi o Gauss-Seidel).

#Método de SOR

In [153]:
def SOR(x0, A, b, w, Tol, Iter):
  D = np.diag(np.diag(A))
  Dinv =np.linalg.inv(D)
  L = np.tril(A, -1)
  U = np.triu(A, 1)
  xk = x0
  n = A.shape[0]

  for k in range(Iter):
    xk1 = (1-w)*xk -(w*Dinv)@(L@xk + U@xk) + w*Dinv@b
    if np.linalg.norm(xk1-xk) < Tol:
      print('El número de iteraciones es k', k)
      print('El error es ||x_k+1 - x_k||: ', np.linalg.norm(xk1-xk))
      return xk1
    else: xk = xk1
  return xk1

In [154]:
#Matrices de interes
A = np.array([[3,-1, 1],
              [3, 6, 2],
              [3, 3, 7]], dtype="float64")

b = np.array([1, 0, 4], dtype="float64")

D = np.diag(np.diag(A))
Dinv =np.linalg.inv(D)
L = np.tril(A, -1)
U = np.triu(A, 1)
T = Dinv@(L+U)
print(T)

[[ 0.         -0.33333333  0.33333333]
 [ 0.5         0.          0.33333333]
 [ 0.42857143  0.42857143  0.        ]]


In [155]:
def p_A(A):
  n = A.shape[0]
  eigvals = np.linalg.eigvals(A)
  alpha = np.zeros(n)

  for i in range(n):
    alpha[i] = np.linalg.norm(eigvals[i])

  return max(alpha)

In [156]:
p_A(T)

0.4193196207983697

In [157]:
np.linalg.eigvals(T)

array([ 0.41931962+0.j       , -0.20965981+0.1132435j,
       -0.20965981-0.1132435j])

$\omega=\frac{2}{1+\sqrt{1-\rho(T)^{2}}}$ donde $T=D^{-1}(L+U)$

In [158]:
#Parametros de la funcion SOR
x0 = np.zeros(A.shape[0], dtype="float64")
Tol = 1e-5
Iter = 1000
w = 2/(1+(1-p_A(T)**2)**(1/2))
print("Omega es", w)

Omega es 1.048306666931363


In [159]:
SOR(x0, A, b, w, Tol, Iter)

El número de iteraciones es k 17
El error es ||x_k+1 - x_k||:  5.673680695816684e-06


array([ 0.0350876 , -0.23684334,  0.65789335])

In [160]:
#Sobrerrelajación
SOR(x0, A, b, 1.1, Tol, Iter)

El número de iteraciones es k 21
El error es ||x_k+1 - x_k||:  6.393273361062588e-06


array([ 0.03508758, -0.23684363,  0.65789303])

In [161]:
#Subrelajación
SOR(x0, A, b, 0.9, Tol, Iter)

El número de iteraciones es k 10
El error es ||x_k+1 - x_k||:  5.173949728875167e-06


array([ 0.03508472, -0.23684167,  0.65789864])

Veamos que con w=0.9 alcanzamos un mínimo de iteraciones.

#Método de Jacobi

In [162]:
def Jacobi(x0, A, b, Tol, Iter):
  D = np.diag(np.diag(A))
  Dinv =np.linalg.inv(D)
  L = np.tril(A, -1)
  U = np.triu(A, 1)
  xk = x0
  n = A.shape[0]

  for k in range(Iter):
    xk1 = -(Dinv)@(L@xk + U@xk) + Dinv@b #Es igual que SOR, pero con w=1
    if np.linalg.norm(xk1-xk) < Tol:
      print('El número de iteraciones es k', k)
      print('El error es ||x_k+1 - x_k||: ', np.linalg.norm(xk1-xk))
      return xk1
    else: xk = xk1
  return xk1

In [163]:
Jacobi(x0, A, b, Tol, Iter)

El número de iteraciones es k 14
El error es ||x_k+1 - x_k||:  5.59448098943512e-06


array([ 0.03508782, -0.23684101,  0.65789596])

Con Jacobi converge más rápido porque este ejercicio converge más rápido con w=1

#Comparación de los métodos con 3 iteraciones

In [164]:
x_solucion = SOR(x0, A, b, w, Tol, Iter)

El número de iteraciones es k 17
El error es ||x_k+1 - x_k||:  5.673680695816684e-06


In [165]:
x_SOR = SOR(x0, A, b, 0.9, Tol, 3)
print("El error es ||x_k+1 - x_k||: ", np.linalg.norm(x_solucion - x_SOR))

El error es ||x_k+1 - x_k||:  0.08200836095461776


In [166]:
x_Jacobi = Jacobi(x0, A, b, Tol, 3)
print("El error es ||x_k+1 - x_k||: ", np.linalg.norm(x_solucion - x_Jacobi))

El error es ||x_k+1 - x_k||:  0.043108821138073086
